# Inférence du modèle de régression linéaire sur une modélisation arpege

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import pickle
import pandas as pd

data = xr.open_dataset('data/arpege__025__SP1__025H048H__2024-04-08T00_00_00Z.grib2', engine='cfgrib') #contient des données H+27 à H+48

In [ ]:
#Visualisation 
x = data['longitude'][:] # 0-359, step = 1
y = data['latitude'][:] # -90~90, step =1
tmp = data['tp'][:][7] # shape(181,360) 

fig = plt.figure(dpi=150)
m = Basemap(projection='mill',lat_ts=10,llcrnrlon=x.min(), urcrnrlon=x.max(),llcrnrlat=y.min(),urcrnrlat=y.max(), resolution='c')

xx, yy = m(*np.meshgrid(x,y))
m.pcolormesh(xx,yy,tmp,cmap=plt.cm.jet,norm=matplotlib.colors.LogNorm(vmin=1,vmax=tmp.values.max(),))
m.colorbar(location='right')

m.drawcoastlines()
m.drawparallels(np.arange(-90.,120.,30.), labels=[1,0,0,0], fontsize=10)
m.drawmeridians(np.arange(0.,360.,60.), labels=[0,0,0,1], fontsize=10)
plt.title("{}, GFS, Temperature (C) ".format(data['time'].values))
plt.show()

In [ ]:
# Récupération des prédictions aux abords des stations
stations_meteo = pd.read_csv('liste_stations_affluents_bordeaux_meteo.csv')
stations_hydro = pd.read_csv('liste_stations_affluents_bordeaux_hydro.csv')

donnees     = pd.read_csv('data/donnees-avril-2024.csv')
donnees = donnees[donnees['Date'] == '2024-04-08'].set_index('Date').fillna(0)

ds = data.get('tp')
ds = ds.to_dataframe()

arpege_j2 = ds.loc['2 days 00:00:00']['tp'].reset_index()
def round_to025(i) :
    return round(i*4)/4

for s in stations_meteo.itertuples() : 
    latitude,longitude = round_to025(s[3]),round_to025(s[4])
    donnees[s[1]] = arpege_j2[(arpege_j2['latitude'] == latitude) & (arpege_j2['longitude'] ==longitude) ]['tp'].values[0]

In [ ]:
# Inférence avec modèle de machine learning
with open('./data/regression-lineaire.pkl','rb') as file :
    model = pickle.load(file)

columns = ['O200001001', 'V504661001', 'O180405201', 'O153291001', 'O353401001',
       'O375401001', 'O546431001', 'O819000201', 'Q066402002', 'O605401001',
       'O633251001', 'K216311001', 'O701581001', 'P171291001', 'O839431001',
       'P257401001', 'Q224291001', '48198001', '65258001_x', '65258001_y',
       '31451001', '65258001', '11221004', '65125001', '12216001']
model.predict(donnees[columns].values)
#Ne marche pas car certaines données sont en trop ou n'existent pas. Problème avec les API Hub'Eau et MeteoFrance, qui donnent des nan